# Configurations

In [9]:
import os
import torch.nn as nn
from torch import device
from federated_learning.nets import MNISTCNN, FashionMNISTCNN, MNISTFFNN
from federated_learning.dataset import MNISTDataset, FashionMNISTDataset
from federated_learning.client.ffnn_client import FFNNClient

In [10]:
class Configuration():
    
    # Dataset Config
    BATCH_SIZE_TRAIN = 132
    BATCH_SIZE_TEST = 1000
    DATASET = MNISTDataset
    
    #MNIST_FASHION_DATASET Configurations
    MNIST_FASHION_DATASET_PATH = os.path.join('./data/mnist_fashion')
    MNIST_FASHION_LABELS = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker',  'Bag', 'Ankle Boot']
    
    #MNIST_DATASET Configurations
    MNIST_DATASET_PATH = os.path.join('./data/mnist')
    
    #CIFAR_DATASET Configurations
    CIFAR10_DATASET_PATH = os.path.join('./data/cifar10')
    CIFAR10_LABELS = ['Plane', 'Car', 'Bird', 'Cat','Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
    
    #Model Training Configurations
    N_EPOCHS = 10
    LEARNING_RATE = 0.01
    MOMENTUM = 0.5
    LOG_INTERVAL = 10
    CRITERION = nn.CrossEntropyLoss
    NETWORK = MNISTFFNN
    NUMBER_TARGETS = 10
    
    #Local Environment Configurations
    NUMBER_OF_CLIENTS = 3
    CLIENT_TYPE = FFNNClient
    DEVICE = device('cpu')
    
    #Label Flipping Attack
    DATA_POISONING_PERCENTAGE = 1
    FROM_LABEL = 5
    TO_LABEL = 4
    
    #Victoria Metrics Configurations
    VM_URL = os.getenv('VM_URL') #URL settings in docker-compose.yml
    

In [69]:
from datetime import datetime
from urllib.request import Request, urlopen
from urllib.error import HTTPError

class VMUtil(): 
    def __init__(self, configs): 
        """
        :param configs: experiment configurations
        :type configs: Configuration
        """
        self.configs = configs
        self.vm_url = "http://"+ configs.VM_URL
        
    def http_request(self, url, data=None):
        """
        Sends a http request with to an url
        :param url: target url
        :type url: string
        :param data: data for a post request
        :type data: string
        return urllib.Request, object
        """
        try:
            request = Request(url, data=data.encode('ascii', 'ignore')) if data else Request(url)
            response = urlopen(request)
            return request, response
        except HTTPError as e: 
            return None, None
            print("ERROR: {}".format(e))

    def get_data_by(self, name):
        """
        Returns all entries of a metrics 
        :param name: metrics __name__
        :type name: string
        return string
        """
        url = self.vm_url +"/api/v1/query?query=%s{}[2y]"% (name)
        request, response = self.http_request(url)
        return response.read().decode("utf-8")
        
    def push_data(self, data):
        """
        Push data to Victoria Metrics Database
        """
        timestamp = int(datetime.timestamp(datetime.now()))
        url = self.vm_url + "/write?precision=s"
        try:
            request, response = self.http_request(url, data=data.format(timestamp))
        except HTTPError as e: 
            print("ERROR: {}".format(e))

In [72]:
configs = Configuration()
vm = VMUtil(configs)

In [73]:
vm.push_data(data = "npn,organization=testorganization,artifactid=testid downloads=1273 {}")

In [74]:
vm.push_data(data = "npm,organization=testorganization,artifactid=testid downloads=1276 {}")

In [75]:
vm.get_data_by("npn_downloads")

'{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"__name__":"npn_downloads","artifactid":"testid","organization":"testorganization"},"values":[[1653342036,"1234"],[1653343239,"1234"],[1653559922,"1234"],[1653560727,"1234"],[1653561830,"1239"],[1653561905,"1251"],[1653563039,"1259"],[1653563039,"1259"],[1653563039,"1259"],[1653563039,"1259"],[1653563948,"1273"],[1653564944,"1273"]]}]}}'

In [76]:
vm.get_data_by("npm_downloads")

'{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"__name__":"npm_downloads","artifactid":"testid","organization":"testorganization"},"values":[[1653561482,"1239"],[1653561797,"1239"],[1653563931,"1267"],[1653564949,"1276"]]}]}}'